# Multi Armed Bandits Epsilon Greedy

Here the idea is not to actually to find the leverage with bigger probability of getting immediate money (ie: CTR).
But the sequence of actions (arm levers) that will maximise the expected future reward.

For example here is not about only chosing the third strategy but the sequence of strategies that gives better reward. Also notice that in real-life you won't have the distributions you want to find (and act) on them.
![alt text](imgs/multi_armed_bandit_testbed.png "Testbed")

On this notebook we will show the simplest MAB method the epsilon greedy, but there are several:
* Epsilon Greedy
* Thompson Sampling
* Upper Confidence Bound 1 (UCB-1)

### Regret
[Regret](https://vwo.com/blog/multi-armed-bandit-algorithm/) is the "number" that represent the following issue:
1. You run a statistical test to undertand your customer behaviour
2. After weeks of test you start to act
3. If you know in advance the result of step 1 and act from the start your profit would be higher

Sometimes the issue of regret is even worst when the customer behaviour change while you are testing.

### A/B testing: How does it work?
In a standard A/B test experiment, we want to measure the likelihood that one variant of a campaign is truly more effective than another, while controlling the probability that our measurement is mistaken:
* either that we think there is a winner when there isn’t
* or that we miss detecting the winning variant. 

In order to make accurate measurements, A/B tests must account for two key values:
* statistical power: Probability that the experiment will actually detect an effect when an effect exists 
* statistical significance: Degree of confidence that the effect we measure is actually present.

A/B testing is a purely exploratory approach. Since contacts are randomly assigned to each variant with equal probability in an A/B test experiment, the overall payoff achievable equals the average payoff of all the variants, and must be lower than that of the winning variant. As illustrated by the example above, a typical A/B test tends to require hundreds of thousands of contacts to reach appropriate statistical power.

### Ok So why MAB
Multi-armed bandit (MAB) algorithms can be thought of as alternatives to A/B testing that balance exploitation and exploration during the learning process. A MAB solution uses existing results from the experiment to allocate more contacts to variants that are performing well, while allocating less traffic to variants that are underperforming. The idea here is the following:
* You will get the insight you want from the A/B test
* While acting optimally when the algorithm start to converge so the regret will always be lower

### References
* [Introduction](https://markelsanz14.medium.com/introduction-to-reinforcement-learning-part-1-multi-armed-bandit-problem-618e8cbf9d4b)
* [Good Article](https://changyaochen.github.io/multi-armed-bandit-mar-2020/)
* [Multi Armed Bandits 101](https://medium.com/opex-analytics/multi-armed-bandits-101-6f4ac62b6bd6)
* [Other methods](https://towardsdatascience.com/multi-armed-bandits-and-reinforcement-learning-dc9001dcb8da)
* [Thompson Sampling](https://towardsdatascience.com/multi-armed-bandits-thompson-sampling-algorithm-fea205cf31df)
* [Upper Confidence Bound](https://towardsdatascience.com/multi-armed-bandits-upper-confidence-bound-algorithms-with-python-code-a977728f0e2d)
* [Comparing Multi-Armed Bandit Algorithms on Marketing Use Cases](https://towardsdatascience.com/comparing-multi-armed-bandit-algorithms-on-marketing-use-cases-8de62a851831)
* [Solving the Multi-Armed Bandit Problem from Scratch in Python](https://www.analyticsvidhya.com/blog/2018/09/reinforcement-multi-armed-bandit-scratch-python/)

In [1]:
import numpy as np

def pull_bandit_arm(bandits, bandit_number):
    """
    Even pulling the arm might be random
    """
    result = np.random.uniform()
    result_binary = result <= bandits[bandit_number]
    return int(result_binary)

def take_epsilon_greedy_action(epsilon, average_rewards_actions):
    result = np.random.uniform()
    if result < epsilon:
        # Explore
        return np.random.randint(0, len(average_rewards_actions)) 
    else:
        # Greedy action.
        return np.argmax(average_rewards_actions) 

### Bit of Hope
Let's pretend we keep track of the CTR of each of our products in a online system

In [2]:
# CTR of each strategy
market_strategies_ctr = [0.1, 0.3, 0.05, 0.55, 0.4]

### Agent Learning

In [3]:
num_iterations = 500
# Actually default value on industry
epsilon = 0.1

# Store info to know which one is the best action in each moment (Kind of value function)
total_rewards = [0 for _ in range(len(market_strategies_ctr))]
total_attempts = [0 for _ in range(len(market_strategies_ctr))]
avg_rewards = [0.0 for _ in range(len(market_strategies_ctr))]

for iteration in range(num_iterations+1):
    action = take_epsilon_greedy_action(epsilon, avg_rewards)
    reward = pull_bandit_arm(market_strategies_ctr, action)
    # Store result.
    total_rewards[action] += reward
    total_attempts[action] += 1
    avg_rewards[action] = total_rewards[action] / float(total_attempts[action])

    if iteration % 100 == 0:
        print('Expected reward at step: {} is {}'.format(
            iteration,['{:.2f}'.format(elem) for elem in avg_rewards]))

Expected reward at step: 0 is ['0.00', '0.00', '0.00', '0.00', '0.00']
Expected reward at step: 100 is ['0.06', '0.00', '0.25', '0.51', '0.20']
Expected reward at step: 200 is ['0.05', '0.42', '0.17', '0.51', '0.17']
Expected reward at step: 300 is ['0.05', '0.43', '0.11', '0.50', '0.12']
Expected reward at step: 400 is ['0.05', '0.40', '0.09', '0.52', '0.12']
Expected reward at step: 500 is ['0.04', '0.40', '0.14', '0.50', '0.10']


### Plot Best Strategy

In [4]:
best_bandit = np.argmax(avg_rewards)
print('\nBest bandit is {} with an mean reward of {:.3f}'.format(
    best_bandit, avg_rewards[best_bandit]))
print('Total mean reward in the {} episodes has been {}'
      .format(num_iterations, sum(total_rewards)))


Best bandit is 3 with an mean reward of 0.498
Total mean reward in the 500 episodes has been 228
